In [ ]:
from aocd.models import Puzzle
from typing import Tuple, Dict
from collections import defaultdict
from itertools import combinations, product, count, chain
from functools import lru_cache, reduce
from operator import mul
import re

puzzle = Puzzle(year=2020, day=16)
print(puzzle.input_data)

In [ ]:
re_rules = re.compile(r'(\w+\s?\w+): (\d+)-(\d+) or (\d+)-(\d+)')
sections = puzzle.input_data.split('\n\n')
# sections = """class: 0-1 or 4-19
# row: 0-5 or 8-19
# seat: 0-13 or 16-19

# your ticket:
# 11,12,13

# nearby tickets:
# 3,9,18
# 15,1,5
# 5,14,9""".split('\n\n')

rules = {}
for line in sections[0].splitlines():
    a,b,c,d,e = re_rules.match(line).groups()
    rules[a] = [(int(b),int(c)), (int(d),int(e))]
yours = [int(x) for x in sections[1].splitlines()[1].split(',')]
nearby = []
for line in sections[2].splitlines()[1:]:
    nearby.append([int(x) for x in line.split(',')])



In [ ]:
 # Part 1
invalids = []
valid_nearby = []
for ticket in nearby:
    ticket_valid = True
    for elem in ticket:
        invalid = (not (rule[0][0] <= elem <= rule[0][1] or rule[1][0] <= elem <= rule[1][1]) for rule in rules.values())
        if all(invalid):
            invalids.append(elem)
            ticket_valid = False
    if ticket_valid:
        valid_nearby.append(ticket)

answer_a = sum(invalids)
print(answer_a)

In [ ]:
puzzle.answer_a = answer_a

In [ ]:
num_fields = len(valid_nearby[0])
rule2idx = {}
idx = -1
laps = 0
rules_available = list(rules.keys())
cols_available = {x:list(range(num_fields)) for x in rules_available}
while len(rules_available) > 0:
    idx = (idx+1)%len(rules_available)
    rule_name = rules_available[idx]
    rule = rules[rule_name]
    if  laps > 10*num_fields:
        break

    
    col_invalid = [(col,any([not (rule[0][0] <= row[col] <= rule[0][1] or rule[1][0] <= row[col] <= rule[1][1]) for row in valid_nearby])) for col in cols_available[rule_name]]
    for i, x in col_invalid:
        if x:
            cols_available[rule_name].remove(i)    
        # col_matches.append(all([rule[0][0] <= row[col] <= rule[0][1] or rule[1][0] <= row[col] <= rule[1][1] for row in valid_nearby]))

    if len(cols_available[rule_name]) == 1:
            rules_available.remove(rule_name)
            col = cols_available[rule_name][0]
            rule2idx[rule_name] = col
            for other in rules_available:
                if rule_name == other:
                    continue
                if col in cols_available[other]:
                    cols_available[other].remove(col)

    if idx == 0:
        laps += 1

if len(rule2idx.keys()) != len(rules.keys()):
    print('Not all rules matched!')        

destination_rules = [x for x in rules.keys() if 'departure' in x]
answer_b = reduce(mul, [yours[rule2idx[x]] for x in destination_rules])

In [ ]:
puzzle.answer_b = answer_b
